In [8]:
FILE_PATH = 'tao-te-ching'
with open(FILE_PATH+'.txt', 'r') as file:
    tao_data = file.read()

In [17]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Marqo
import marqo

In [4]:
!pip install -qU langchain-text-splitters

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [10]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

In [11]:
texts = text_splitter.split_documents([tao_data])
print(texts[0])
print(texts[1])

page_content='1\nThe Tao that can be trodden is not the enduring and\nunchanging Tao. The name that can be named is not the enduring and\nunchanging name.\n\n(Conceived of as) having no name, it is the Originator of heaven\nand earth; (conceived of as) having a name, it is the Mother of all\nthings.\n\nAlways without desire we must be found,\nIf its deep mystery we would sound;\nBut if desire always within us be,\nIts outer fringe is all that we shall see.'
page_content='Under these two aspects, it is really the same; but as development\ntakes place, it receives the different names. Together we call them\nthe Mystery. Where the Mystery is the deepest is the gate of all that\nis subtle and wonderful.\n\n2\nAll in the world know the beauty of the beautiful, and in doing\nthis they have (the idea of) what ugliness is; they all know the skill\nof the skilful, and in doing this they have (the idea of) what the\nwant of skill is.'


In [12]:
len(texts)

156

In [18]:
# initialize marqo
marqo_url = "http://localhost:8882"  # if using marqo cloud replace with your endpoint (console.marqo.ai)
marqo_api_key = ""  # if using marqo cloud replace with your api key (console.marqo.ai)

client = marqo.Client(url=marqo_url, api_key=marqo_api_key)

In [21]:
client.delete_index("langchain-demo")

{'acknowledged': True}

In [22]:
index_name = "langchain-demo"

docsearch = Marqo.from_documents(texts, index_name=index_name)

Index langchain-demo exists.


KeyError: 'index_defaults'

# qDrant

In [40]:
from datasets import load_dataset

dataset = load_dataset("infoslack/mistral-7b-arxiv-paper-chunked", split="train")
data = dataset.to_pandas()
docs = data[['chunk', 'source']]


In [41]:
from langchain_community.document_loaders import DataFrameLoader

loader = DataFrameLoader(docs, page_content_column="chunk")
documents = loader.load()

In [43]:
from langchain_qdrant import Qdrant
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)
#doc_store = Qdrant.from_texts(
#    documents, embeddings, url="localhost:6333", api_key="", collection_name="texts"
#)


qdrant = Qdrant.from_documents(
    documents=documents,
    embedding=embeddings,
    url='localhost:6333',
    collection_name="chatbot",
    api_key=''
)


ResponseHandlingException: [SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1006)